In [1]:
!pip install folium
import requests

import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0
import folium 

from collections import Counter
from bs4 import BeautifulSoup


In [2]:
CLIENT_ID = 'VRQGHBQTFZX2WAN1BSAJ14NM5MNTY3C304TOFPHB2T3Q5DXX' # Foursquare ID
CLIENT_SECRET = '1N02DDZ1II2WG1BBMUA23L5ZIKPNRSK2LOTPE1DH00VBILOE' # Foursquare Secret
VERSION = '20180604'
LIMIT = 50
radius = 500

<h3> Get 30 most popular venue categories from each location in Foursquare API </h3>

In [3]:
locations = ["New York City, NY", "San Francisco, CA", "Chicago, IL", "Boston, MA", "Philadelphia, PA"]
popular_venue_data = {}
for location in locations:
    location_data = {}
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&radius={}&limit={}&sortByPopularity=1'.format(CLIENT_ID, CLIENT_SECRET, location, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    for venue in results['response']['groups'][0]['items']:
        venue_type = venue['venue']['categories'][0]['name']
        if venue_type in location_data:
            location_data[venue_type] += 1
        else:
            location_data[venue_type] = 1
    popular_venue_data[location]=location_data


<h2> Get 5 most popular venues and insert data into Pandas DataFrame </h2>

<h6> Sorted in order from Most Popular (1) to Least Popular (5)</h6>

In [4]:
data = {'City':[], 1:[], 2:[], 3:[], 4:[], 5:[]}
for loc in locations:
    popular_venue_data[loc] = [i[0] for i in Counter(popular_venue_data[loc]).most_common(5)]
    data['City'].append(loc)
    for i in range(1,6):
        data[i].append(popular_venue_data[loc][i-1])
popular_venue_df=pd.DataFrame(data)
popular_venue_df["City"].replace({"New York City, NY":"NYC", "San Francisco, CA":"San Francisco", "Chicago, IL":"Chicago", "Boston, MA":"Boston", "Philadelphia, PA":"Philadelphia"}, inplace=True)

<H1> DataFrame of the Population Density in Each Location </H1>

<h3> Scraped from governing.com using BeautifulSoup and data entered into a Pandas DF</h3>

In [5]:
res = requests.get("https://www.governing.com/gov-data/population-density-land-area-cities-map.html")
soup = BeautifulSoup(res.content, 'lxml')
tables = soup.find_all('table')
locations = ["New York, New York", "San Francisco", "Chicago", "Boston", "Philadelphia"]
for table in tables:
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        for elem in locations:
            if len(row)>1 and elem in row[0] :
                res.append(row)
                locations.remove(elem)
                break
                
population_density_df = pd.DataFrame(res, columns=['City', 'Population Density (Persons/Square Mile)', '2016 Population', 'Land Area (Square Miles)'])
population_density_df["City"].replace({"New York, New York":"NYC", "San Francisco, California":"San Francisco", "Chicago, Illinois":"Chicago", "Boston, Massachusetts":"Boston", "Philadelphia, Pennsylvania":"Philadelphia"}, inplace=True)

<H1> DataFrame of Median Home Price in Each Location </H1>

<h3> Scraped from kiplinger.com using BeautifulSoup and data entered into a Pandas DF</h3>

In [6]:
res = requests.get("https://www.kiplinger.com/article/real-estate/T010-C000-S002-home-price-changes-in-the-100-largest-metro-areas.html")
soup = BeautifulSoup(res.content, 'lxml')
tables = soup.find_all('table')
locations = ["New York, N.Y.", "San Francisco", "Chicago", "Boston", "Philadelphia"]
for table in tables:
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        for elem in locations:
            if len(row)>1 and elem in row[0] :
                res.append(row)
                locations.remove(elem)
                break
                
home_prices_df = pd.DataFrame(res, columns=['Metro Area', 'Median Home Price($)', '% Change 1 Year', '% Change Since Peak', "% Change Since Bottom", "Affordability Index"])
home_prices_df["Metro Area"].replace({"New York, N.Y.-N.J.":"NYC", "San Francisco, Calif.":"San Francisco", "Chicago, Ill.":"Chicago", "Boston, Mass.":"Boston", "Philadelphia, Pa.":"Philadelphia"}, inplace=True)

<H1> DataFrame of the School District Size in Each Location </H1>

<h3> Scraped from wikipedia.com using BeautifulSoup and data entered into a Pandas DF</h3>

In [7]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_the_largest_school_districts_in_the_United_States_by_enrollment")
soup = BeautifulSoup(res.content, 'lxml')
tables = soup.find_all('table')
locations = ["New York City", "San Francisco", "Chicago", "Boston", "Philadelphia"]
for table in tables:
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        for elem in locations:
            if len(row)>1 and elem in row[2] :
                res.append(row)
                locations.remove(elem)
                break
                
school_df = pd.DataFrame(res, columns=['Rank', 'vs 2014', 'School District', "State/Territory", "Students in Autumn of 2016"])
school_df["School District"].replace({"New York City":"NYC", "San Francisco Unified":"San Francisco", "City of Chicago (SD 299)":"Chicago", "Philadelphia City":"Philadelphia"}, inplace=True)


<h1> Compile All Data from previous dfs into a single df </h1>

In [14]:
data = {'City':[], "School District Size":[], "Population Density (per sq mi)":[], "Most Popular Venue":[], "Venue 2":[], "Venue 3":[], "Venue 4":[], "Venue 5":[]}
locations = ["NYC", "San Francisco", "Chicago", "Philadelphia", 'Boston']

for loc in locations:
    data['City'].append(loc)
    data['School District Size'].append(school_df['Students in Autumn of 2016'][school_df['School District']== loc].iloc[0])
    data['Population Density (per sq mi)'].append(population_density_df['Population Density (Persons/Square Mile)'][population_density_df['City']==loc].iloc[0])
    data['Most Popular Venue'].append(popular_venue_df[1][popular_venue_df['City']==loc].iloc[0])
    data['Venue 2'].append(popular_venue_df[2][popular_venue_df['City']==loc].iloc[0])
    data['Venue 3'].append(popular_venue_df[3][popular_venue_df['City']==loc].iloc[0])
    data['Venue 4'].append(popular_venue_df[4][popular_venue_df['City']==loc].iloc[0])
    data['Venue 5'].append(popular_venue_df[5][popular_venue_df['City']==loc].iloc[0])

overall_df = pd.DataFrame(data)
overall_df

,City,School District Size,Population Density (per sq mi),Most Popular Venue,Venue 2,Venue 3,Venue 4,Venue 5
0,NYC,"984,462","28,211",American Restaurant,Gym / Fitness Center,Coffee Shop,Grocery Store,Hotel
1,San Francisco,"60,133","18,581",Coffee Shop,Café,Cocktail Bar,New American Restaurant,Food & Drink Shop
2,Chicago,"378,199","11,883",Mexican Restaurant,Train Station,Dive Bar,Donut Shop,Brewery
3,Philadelphia,"133,929","11,692",Hotel,Plaza,Grocery Store,Convenience Store,Coffee Shop
4,Boston,"53,640","13,943",American Restaurant,Historic Site,Coffee Shop,Pub,Market


<h2> One-hot Encoding </h2> <br> 
    <em><b>**** NOTE: Since these cities are not just being compared for similarity, the Median Home Price is NOT included in clustering. This is because we are also looking for a more economically-friendly location. Based on similarity, we will then use home price to see which city has cheaper real estate but still has a similar population to NYC***</b></em>

In [9]:
overall_onehot = pd.get_dummies(data=overall_df, columns=['School District Size', 'Population Density (per sq mi)','Most Popular Venue', 'Venue 2', 'Venue 3',
       'Venue 4', 'Venue 5'], prefix="", prefix_sep="")
overall_onehot

,City,"133,929","378,199","53,640","60,133","984,462","11,692","11,883","13,943","18,581",...,Convenience Store,Donut Shop,Grocery Store,New American Restaurant,Pub,Brewery,Coffee Shop,Food & Drink Shop,Hotel,Market
0,NYC,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,San Francisco,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
2,Chicago,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,Philadelphia,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,Boston,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


In [10]:
overall_onehot.shape

(5, 34)

<h2> K-Means Clustering </h2>

<h6> Repeated with values of k between 0 and 5 </h6>

In [11]:
# set number of clusters
for i in range(1,5):
    kclusters = i

    clustering = overall_onehot.drop('City',1)
    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

    # check cluster labels generated for each row in the dataframe
    kmeans.labels_[0:10] 
    # add clustering labels
    overall_df['Cluster Labels'] = kmeans.labels_
    print("\n\n Number of clusters:", i)
    print(overall_df[['Cluster Labels', 'City']])



 Number of clusters: 1
   Cluster Labels           City
0               0            NYC
1               0  San Francisco
2               0        Chicago
3               0   Philadelphia
4               0         Boston


 Number of clusters: 2
   Cluster Labels           City
0               1            NYC
1               0  San Francisco
2               0        Chicago
3               0   Philadelphia
4               1         Boston


 Number of clusters: 3
   Cluster Labels           City
0               0            NYC
1               2  San Francisco
2               1        Chicago
3               0   Philadelphia
4               0         Boston


 Number of clusters: 4
   Cluster Labels           City
0               1            NYC
1               0  San Francisco
2               3        Chicago
3               2   Philadelphia
4               1         Boston


<h2> Clustering Isolate Data from NYC cluster (using k=3) and add back Median Home Price Data </h2>

In [12]:
kclusters = 3
clustering = overall_onehot.drop('City',1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
overall_df['Cluster Labels'] = kmeans.labels_
NYC_cluster = overall_df[overall_df['Cluster Labels']==(overall_df[overall_df['City']=='NYC']['Cluster Labels'].iloc[0])]
NYC_cluster.drop(['Cluster Labels'],axis=1,inplace=True)
NYC_cluster

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,City,School District Size,Population Density (per sq mi),Most Popular Venue,Venue 2,Venue 3,Venue 4,Venue 5
0,NYC,"984,462","28,211",American Restaurant,Gym / Fitness Center,Coffee Shop,Grocery Store,Hotel
3,Philadelphia,"133,929","11,692",Hotel,Plaza,Grocery Store,Convenience Store,Coffee Shop
4,Boston,"53,640","13,943",American Restaurant,Historic Site,Coffee Shop,Pub,Market


In [13]:
def AddMedianHomeValue(row):
    home_price = home_prices_df['Median Home Price($)'][home_prices_df['Metro Area']==row['City']]
    return home_price.iloc[0]

NYC_cluster['Median Home Price ($)'] = NYC_cluster.apply(lambda row: AddMedianHomeValue(row),axis=1)
NYC_cluster

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,City,School District Size,Population Density (per sq mi),Most Popular Venue,Venue 2,Venue 3,Venue 4,Venue 5,Median Home Price ($)
0,NYC,"984,462","28,211",American Restaurant,Gym / Fitness Center,Coffee Shop,Grocery Store,Hotel,"418,000"
3,Philadelphia,"133,929","11,692",Hotel,Plaza,Grocery Store,Convenience Store,Coffee Shop,"220,000"
4,Boston,"53,640","13,943",American Restaurant,Historic Site,Coffee Shop,Pub,Market,"421,000"


<h1> Conclusion/Analysis </h1>

Given that Boston is clustered with NYC in almost all k-values, it is clear that Boston is <em> most similar </em> in population density, school district size, venue popularity, etc. to NYC. 

However, the median home price is actually higher in Boston than NYC. While Philadelphia is less similar to NYC than Boston, it is more similar to NYC than Chicago or San Francisco and has half the real estate price of NYC. 

In conclusion: in most cases, <b>Philadelphia</b> would be the ideal choice for start-ups looking to have a similar audience to NYC while having to pay less in real estate 